Credit to https://github.com/zhixuhao/unet for the Unet Model

# Imports

In [ ]:

# !pip install rasterio #used in Google Colab to install the library
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
%tensorflow_version 1.x magic
import numpy as np
import os
import rasterio
import keras
import tensorflow as tf
import random
import pandas as pd
# from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, GlobalMaxPool2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import time
import skimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

'''Lines used in Google Colab to load files'''
# from google.colab import drive
# from google.colab import files
# drive.mount('/content/gdrive')

# Functions (normalize, create_dataset, image_tiles)

In [ ]:
def normalize(array):
  return (array - array.min()) / (array.max() - array.min())

def create_dataset(im, dim_size,stride):
    
  no_bands = len(im.indexes)
  data = im.read()
  bands = np.zeros([no_bands,im.shape[0],im.shape[1]])
  for k in range(no_bands):
    temp = data[k,:,:]
    bands[k,:,:] = normalize(temp)
      
  global rows
  global columns
  rows = int((im.shape[0]-dim_size)/stride) + 1
  columns = int((im.shape[1]-dim_size)/stride) + 1

  dataset= np.zeros((rows*columns,dim_size,dim_size,no_bands))
  # dataset = np.float16(dataset)
  image_num = 0 
  y_min = 0
  y_max = dim_size
  global orig_width
  global orig_height
  orig_height = im.shape[0]
  orig_width = im.shape[1]
  while (y_max)<orig_height:
    x_min = 0
    x_max = dim_size
    while (x_max)<orig_width:
      for k in range(no_bands):
        temp = bands[k,y_min:y_max,x_min:x_max]
        dataset[image_num,:,:,k] = temp
      # plt.figure()
      # plt.imshow(dataset[image_num,:,:,k],cmap='gray')
      # plt.draw()
      image_num = image_num + 1
      x_min = x_min + stride
      x_max = x_max + stride
        
    y_min = y_min + stride
    y_max = y_max + stride
  # dataset = np.float16(dataset)
  return dataset,no_bands

def image_tiles(path, image_size, stride,mask_path):
  # Creating tiles for images
  flag = True
  # print("Processed " + f)
  im = rasterio.open(path)
  mask = rasterio.open(mask_path)
  [tempy, no_labels] = create_dataset(mask, image_size, stride)
  [tempx, no_bands] = create_dataset(im, image_size, stride)
  # tempy = np.float16(tempy)
  # tempx = np.float16(tempx)
  if flag:
    flag = False
    x = tempx
    y = tempy
  else:
    x = np.concatenate([x, tempx],0)
    y = np.concatenate([y, tempy],0)
    # x = np.float16(x)
    # y = np.float16(y)

            
  plt.figure(figsize=(15,15))
  plt.imshow(im.read()[0,:,:],cmap='gray')
  # plt.figure(figsize=(15,15))
  # plt.imshow(mask.read()[0,:,:],cmap='gray')
  print('Number of images generated: {} \nNumber of labels generated: {}'.format(x.shape[0],y.shape[0]))
  del tempx
  del tempy
  del im
  del mask
  return x, y, no_bands, no_labels


# Dataset Creation

In [ ]:
image_size = 64 # New image size
stride = 32 # Stride

file_name = "dem_rgbn_clipped_lidar.tif"
# file_name = "elev_rgbn_clipped_lidar.tif"
# file_name = "rgbn_clipped_lidar.tif"
# file_name = "rgb_clipped_lidar.tif"
area_size = "big"

train_image_path = "./gdrive/My Drive/Jupyter Dataset/Lidar_Class/"+area_size+"/training_" + file_name
test_image_path = "./gdrive/My Drive/Jupyter Dataset/Lidar_Class/"+area_size+"/validation_" + file_name

[x_train, y_train, no_bands, no_labels] = image_tiles(train_image_path,image_size,stride,"./gdrive/My Drive/Jupyter Dataset/Lidar_Class/"+area_size+"/training_envi_final_mask.tif")
[x_valid, y_valid, no_bands, no_labels] = image_tiles(test_image_path,image_size,stride,"./gdrive/My Drive/Jupyter Dataset/Lidar_Class/"+area_size+"/validation_envi_final_mask.tif")
no_classes = no_labels

# U-net Model

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def get_unet(input_img, no_classes, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(no_classes, (1, 1), activation='softmax') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

# Model Compilation and Parameter selection

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR) # Supressing Warnings
input_img = Input((image_size, image_size, no_bands), name='img')
model = get_unet(input_img, no_labels, n_filters=16, dropout=0.25, batchnorm=True)
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

# Model Training 

In [ ]:
epochs = 30
batch_size = 64
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_valid, y_valid))

# Accuracy Plot

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
ax = plt.figure(figsize = (16,6)).gca()
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim((0,1))
plt.xlim((0,epochs))
ax.xaxis.set_major_locator(MaxNLocator(nbins=epochs/5, integer=True))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.savefig('plot_1.png', bbox_inches='tight',dpi=300)

# Loss Plot

In [ ]:
plt.style.use('seaborn')
plt.tight_layout()
ax = plt.figure(figsize = (16,6)).gca()
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
# plt.ylim((0,1))
plt.xlim((0,epochs))
ax.xaxis.set_major_locator(MaxNLocator(nbins=epochs/5, integer=True))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.savefig('plot_2.png', bbox_inches='tight',dpi=300)

# Defining Image Reconstruction

In [ ]:
def image_reconstruction(data, dim_size, stride, rows=rows, columns=columns):
  no_original_images = int(data.shape[0]/(rows*columns))
  var1 = data.shape[0]
  original_images = {}
  im_num = 0
  if dim_size == stride:
    for im in range(no_original_images):
      og_image= []
      flag2 = True
      for c in range(rows):
        row = []
        flag = True
        for p in range(columns):
          # plt.figure()
          # plt.imshow(data[im_num,:,:,0],cmap='gray')
          if flag:
            temp = data[im_num,:,:,:]
            row = temp
            flag = False
          else:
            temp = data[im_num,:,:,:]
            row = np.concatenate((row,temp), axis=1)
          im_num = im_num + 1
        if flag2:
          og_image = row
          flag2 = False
        else:
          og_image = np.concatenate((og_image, row), axis=0)
      original_images[im] = og_image

  else:
    for im in range(no_original_images):
      og_image= []
      y_max=dim_size
      while y_max<orig_height and im_num<var1:
          row = []
          x_max=dim_size
          flag=True
          while x_max<orig_width and im_num<var1:
            if flag==True and y_max==dim_size and x_max==dim_size:
              temp = data[im_num,:,:,:]
              row = temp
              flag=False
            if flag==True and y_max>dim_size:
              temp = data[im_num,stride:,:,:]
              row = temp
              flag=False
            if flag==False and y_max==dim_size and x_max>dim_size:
              temp = data[im_num,:,stride:,:]
              row = np.concatenate((row,temp), axis=1)
            if flag==False and y_max>dim_size and x_max>dim_size:
              temp = data[im_num,stride:,stride:,:]
              row = np.concatenate((row,temp), axis=1)
            x_max = x_max + stride
            im_num = im_num + 1
          if y_max==dim_size:
            og_image = row
          else:
            og_image = np.concatenate((og_image, row), axis=0)
          y_max = y_max + stride
          original_images[im] = og_image

  return original_images

# Prediction and image reconstruction

In [ ]:
result = model.predict(x_valid)
for i in range(no_labels):
  result[:,:,:,i] = result[:,:,:,i] > 0.5
result_final = image_reconstruction(result, image_size, stride)
y_valid_final = image_reconstruction(y_valid, image_size, stride)

# Plotting predictions for every class 

In [ ]:

im_result = result_final[0] #prediction for the first image in the test dataset
im_y_valid = y_valid_final[0]

plt.style.use('default')
fig, ax = plt.subplots(2,no_labels,figsize=(25,20))

# class_names = ['Buildings with tiling','Road',
#               'Sparse Vegetation','Forests',
#               'Buildings w/o tiling',
#               'Bare Land','Sea','No Data' ] # first ground truth

class_names = ['Buildings',
               'No Data',
              'Sparse Vegetation','Bare Land','Rocks',
               'Sea','Forests',
              'Roads'
               ]# second ground truth

for i in range(no_labels):
  ax[0,i].set_title('Pred. - '+ class_names[i])
  ax[0,i].imshow(im_result[:,:,i], cmap='gray')
  ax[0,i].axis('off')
  ax[1,i].set_title('Orig. - '+ class_names[i])
  ax[1,i].imshow(im_y_valid[:,:,i], cmap='gray')
  ax[1,i].axis('off')
# plt.tight_layout()
fig.savefig('plot_3', bbox_inches='tight',dpi=600)

# Plotting validation predictions 

In [ ]:
#@title Validation Plot
im_result_colors = np.zeros((im_result.shape[0],im_result.shape[1]))
im_y_valid_colors = np.zeros((im_y_valid.shape[0],im_y_valid.shape[1]))
for k in range(im_result.shape[2]):
  im_result_colors = im_result_colors + im_result[:,:,k]*k
  im_y_valid_colors = im_y_valid_colors + im_y_valid[:,:,k]*k
plt.figure(figsize=(20,20))

'''First ground truth'''
# cmap = matplotlib.colors.LinearSegmentedColormap.from_list("",
# [
# matplotlib.colors.to_rgb([1,0,0]), # 'Disc. Urban Fabric'
#  matplotlib.colors.to_rgb([0.8,0.8,0.8]), # 'Roads'
#  matplotlib.colors.to_rgb([0.8,1,0.8]), # 'Sparsely Veg. Areas'
#  matplotlib.colors.to_rgb([0,0.650980392,0]), # 'Coniferous Forest'
#  matplotlib.colors.to_rgb([1,1,1]), # 'Building without roof' 
# "sandybrown", # Land/soil/ground
#  "lightblue", # 'Sea'
#  matplotlib.colors.to_rgb([0,0,0]) # 'No Data'
# ])

'''Second ground truth'''
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("",
[
matplotlib.colors.to_rgb([1,0,0]), # 'Disc. Urban Fabric'
matplotlib.colors.to_rgb([0,0,0]), # 'No Data'
matplotlib.colors.to_rgb([0.8,1,0.8]), # 'Sparsely Veg. Areas'
"sandybrown", # Land/soil/ground
matplotlib.colors.to_rgb([0.8,0.8,0.8]), # 'Rocks'
"lightblue", # 'Sea'
matplotlib.colors.to_rgb([0,0.650980392,0]), # 'Coniferous Forest'
matplotlib.colors.to_rgb([1,1,1]), # 'Roads'
])

plt.imshow(im_result_colors, cmap=cmap)
plt.colorbar()
plt.savefig('plot_4', bbox_inches='tight',dpi=300)

# Confusion Matrix Function

In [ ]:
def calc_conf_mat(px_min,px_max,py_min,py_max,im_result=im_result,im_y_valid=im_y_valid,class_names=class_names):
  conf_mat = np.zeros((im_result.shape[2],im_y_valid.shape[2]),dtype=int)
  result_classes = np.argmax(im_result,axis=2)
  y_valid_classes = np.argmax(im_y_valid,axis=2)
  test_counter = 0
  for y in range(px_min,px_max):
    for x in range(py_min,py_max):
      conf_mat[y_valid_classes[x,y], result_classes[x,y]] = conf_mat[y_valid_classes[x,y],result_classes[x,y]]+ 1
      test_counter += 1
  df = pd.DataFrame(conf_mat, columns=class_names, index=class_names)
  temp = []
  for i in df:
    temp.append(df[i].sum())
  temp2 = []
  for i in range(conf_mat.shape[0]):
    temp2.append(conf_mat[i].sum())
  temp2.append(np.sum(temp2))
  df.loc['Total'] = temp
  df['Total'] = temp2
  # print("Overall Accuracy: %.4f"%(accuracy))
  return df

# Declaring polygons - Calculating Confusion Matrix

In [ ]:
px1_min, py1_min = (50,500)
px1_max, py1_max = (px1_min +200, py1_min +200)
p1 = [[px1_max,py1_max], [px1_min,py1_max], [px1_min,py1_min], [px1_max,py1_min],[px1_max,py1_max]]
px2_min, py2_min = (200,2905)
px2_max, py2_max = (px2_min +75, py2_min +150)
p2 = [[px2_max,py2_max], [px2_min,py2_max], [px2_min,py2_min], [px2_max,py2_min],[px2_max,py2_max]]
px3_min, py3_min = (200,1000)
px3_max, py3_max = (px3_min +100, py3_min +100)
p3 = [[px3_max,py3_max], [px3_min,py3_max], [px3_min,py3_min], [px3_max,py3_min],[px3_max,py3_max]]
px4_min, py4_min = (660,400)
px4_max, py4_max = (px4_min +100, py4_min +100)
p4 = [[px4_max,py4_max], [px4_min,py4_max], [px4_min,py4_min], [px4_max,py4_min],[px4_max,py4_max]]

df1	  =	calc_conf_mat(	px1_min		,	px1_max		,	py1_min		,	py1_max		)
df2	  =	calc_conf_mat(	px2_min		,	px2_max		,	py2_min		,	py2_max		)
df3	  =	calc_conf_mat(	px3_min		,	px3_max		,	py3_min		,	py3_max		)
df4	  =	calc_conf_mat(	px4_min		,	px4_max		,	py4_min		,	py4_max		)


df_final = df1 + 	df2 + df3 + df4
df_final


# Omission/Commission Errors - Total Accuracy

In [ ]:
counter = 0
diagonal = 0
for i in df_final:
  if i == 'Total':
    break
  else:
    diagonal = diagonal + df_final[i][i]
  if df_final[i][-1] == 0:
    commission_error = 0
  else:
    commission_error = (df_final[i][-1] - df_final[i][counter]) / df_final[i][-1]
  if df_final.loc[i][-1] == 0:
    omission_error = 0
  else:
    omission_error = (df_final.loc[i][-1] - df_final.loc[i][counter]) / df_final.loc[i][-1]
  counter += 1
  
  print('%s\nOmission Error: %.1f %% , Commission Error: %.1f %% \n'%(i, omission_error*100,commission_error*100))
accuracy = diagonal/df_final['Total']['Total']
print("Overall Accuracy: %.1f %%"%(accuracy*100))

# Structural Similarity Index

In [ ]:
temp = y_valid_final[0][:,:,-1]
indices = temp<1
for i in y_valid_final:
  tim1 = y_valid_final[i].astype(float)
  tim2 = result_final[i].astype(float)
  total = skimage.metrics.structural_similarity(tim1, tim2)
  tim3 = y_valid_final[i][indices,:].astype(float)
  tim4 = result_final[i][indices,:].astype(float)
  inner = skimage.metrics.structural_similarity(tim3, tim4)
  print("Image",i+1)
  print("Total similarity: %.4f\nInner similarity: %.4f\n"%(total,inner))

# Calculating weight significance

In [ ]:
#@title Weight significance
filters, biases = model.layers[1].get_weights()
print(filters.shape)
for i in range(filters.shape[2]):
  print(np.sum(abs(filters[:,:,i,:])))

# Plot image with Conf. Matrix polygons

In [ ]:
#@title Plot
im_result_colors = np.zeros((im_result.shape[0],im_result.shape[1]))
im_y_valid_colors = np.zeros((im_y_valid.shape[0],im_y_valid.shape[1]))
for k in range(im_result.shape[2]):
  im_result_colors = im_result_colors + im_result[:,:,k]*k
  im_y_valid_colors = im_y_valid_colors + im_y_valid[:,:,k]*k

fig = plt.figure(1, figsize=(20, 20))
ax = plt.gca()
im = ax.imshow(im_result_colors, cmap=cmap)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
plt.colorbar(im, cax=cax)

xp1	  ,	yp1	  =	zip(	*p1	  )
xp2	  ,	yp2	  =	zip(	*p2	  )
xp3	  ,	yp3	  =	zip(	*p3	  )
xp4	  ,	yp4	  =	zip(	*p4	  )

ax.plot(	xp1	,	yp1	,	'r-'	)
ax.plot(	xp2	,	yp2	,	'r-'	)
ax.plot(	xp3	,	yp3	,	'r-'	)
ax.plot(	xp4	,	yp4	,	'r-'	)